### Setup, Install, and import required libraries

In [1]:
import os
import sys
import re
import numpy as np
import pandas as pd
from csv import QUOTE_NONE
import math
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import ParameterGrid
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

#### IMPORTANT NOTE: THROUGHOUT THE PROGRAM, WE WILL CONSIDER CLICKBAIT AS '1' AND NOT-CLICKBAIT as '0'.

#### Reading the Data

#### Tasks
1) Read the clickbait data and the not clickbait data.
2) Merging the two datasets.
3) Shuffling the data after merging.
4) Splitting the data into train-validation-test split.

In [2]:
#Reading clickbait dataset.
string = []    
with open("clickbait", "r") as f:
    all_lines = f.read()
    for line in re.split(r"(\n)", all_lines):
        if line != "\n":
            string.append(line)

df_clickbait = pd.DataFrame(string[0:len(string)-1],columns=["Text"])
df_clickbait

,Text
0,"Man repairs fence to contain dog, hilarity ens..."
1,Long-Term Marijuana Use Has One Crazy Side Eff...
2,The water from his ear trickles into the bucke...
3,You'll Never Guess What Nick Jonas Does in the...
4,How Cruise Liners Fill All Their Unsold Cruise...
...,...
809,Mark your calendars: Jon Stewart announces the...
810,OITNB's Taylor Schilling and Carrie Brownstein...
811,Researchers have discovered the average penis ...
812,Why it may be smart to wait to put on sunscree...


In [3]:
#Reading not-clickbait dataset.
string = []    
with open("not-clickbait", "r") as f:
    all_lines = f.read()
    for line in re.split(r"(\n)", all_lines):
        if line != "\n":
            string.append(line)

df_not_clickbait = pd.DataFrame(string[0:len(string)-1],columns=["Text"])
df_not_clickbait

,Text
0,Congress Slips CISA Into a Budget Bill That's ...
1,DUI Arrest Sparks Controversy
2,It’s unconstitutional to ban the homeless from...
3,A Government Error Just Revealed Snowden Was t...
4,A toddler got meningitis. His anti-vac parents...
...,...
1569,US releases Guantánamo prisoner after 14 years...
1570,Loophole means ecstasy and loads of other drug...
1571,Astronomers Watch a Supernova and See Reruns
1572,"In Indian Rapists’ Neighborhood, Smoldering An..."


In [4]:
#Adding a label in the two datasets we read above.

df_clickbait['label_code'] = 1
df_not_clickbait['label_code'] = 0

In [5]:
#Merging the two datasets.
data = pd.concat([df_clickbait, df_not_clickbait], ignore_index=True)

In [6]:
#Shuffling the data to randomize.
data = data.sample(frac=1).reset_index(drop=True)

In [7]:
#Splitting the data into train-validation-test datasets.
train_text, test_text, train_label_code , test_label_code = train_test_split(data['Text'], data['label_code'], test_size = 0.20, random_state=1)
train_text, validation_text, train_label_code, validation_label_code = train_test_split(train_text, train_label_code, test_size = 0.1, random_state=1)

print('Training set Percentage:', round(100 * len(train_text)/len(data),0))
print('Validation set Percentage:', round(100 * len(validation_text)/len(data),0))
print('Testing set Percentage:', round(100 * len(test_text)/len(data),0))

print("")

print('Training set distribution:\n', train_label_code.value_counts())
print('Validation set distribution:\n', validation_label_code.value_counts())
print('Testing set distribution:\n', test_label_code.value_counts())

Training set Percentage: 72.0
Validation set Percentage: 8.0
Testing set Percentage: 20.0

Training set distribution:
 0    1141
1     578
Name: label_code, dtype: int64
Validation set distribution:
 0    123
1     68
Name: label_code, dtype: int64
Testing set distribution:
 0    310
1    168
Name: label_code, dtype: int64


In [8]:
#Getting the value counts of each class to compute the target rate.
train_not_clickbait, train_clickbait = train_label_code.value_counts()
validation_not_clickbait, validation_clickbait = validation_label_code.value_counts()
test_not_clickbait, test_clickbait = test_label_code.value_counts()

#Computing the target rate of the datasets.
train_target_rate = round(100 * train_clickbait / (train_clickbait + train_not_clickbait),0)
validation_target_rate = round(100 * validation_clickbait / (validation_clickbait + validation_not_clickbait),0)
test_target_rate = round(100 * test_clickbait / (test_clickbait + test_not_clickbait),0)

#Printing the target rates of the datasets.
print("Target rate of Training dataset:", train_target_rate,"%")
print("Target rate of Validation dataset:", validation_target_rate,"%")
print("Target rate of Test dataset:", test_target_rate,"%")

Target rate of Training dataset: 34.0 %
Target rate of Validation dataset: 36.0 %
Target rate of Test dataset: 35.0 %


<b>COMMENTS: The target rate looks pretty similar in all the datasets which is a good sign. All the datasets are imbalanced that is a different story. The not-clickbait samples are more than the clickbait samples in all the datasets which might be a bias problem.

#### Baseline Performance

Assuming if we have a trivial baseline classifer that flags every text presented as clickbait, the precision, recall, and F1-score are as follows:
<br><br>The target rate for my testing dataset is 35.0% which means the baseline classifier will classify these correctly.
<br>So, True Positives = 0.35 or 35%
<br>The remaining cases of the test dataset will also be classifed as Positive which in actual were Negative.
<br>So, False Positives = 0.65 or 65%
<br>
<br>As the classifier does not classifies any text as Negative, the False Negatives = 0 and the True Negatives = 0.
<br>
<br>Precision is a measure of how many of the positive predictions made are correct (true positives). The formula for it is:
<br>Precision = (True Positives) / (True Positives + False Positives) = (0.35) / (0.35 + 0.65) = 0.35
<br><br>Recall is a measure of how many of the positive cases the classifier correctly predicted, over all the positive cases in the data. The formula for it is:
<br>Recall  = (True Positives) / (True Positives + False Negatives) = (0.35) / (0.35 + 0) = 1.00
<br><br>F1-Score is a measure combining both precision and recall. It is generally described as the harmonic mean of the two. Harmonic mean is just another way to calculate an “average” of values, generally described as more suitable for ratios (such as precision and recall) than the traditional arithmetic mean. The formula used for F1-score in this case is:
<br>F1-Score = (2 * Precision * Recall) / (Precision + Recall) = (2 * 0.35 * 1) / (0.35 + 1) = (0.70) / (1.35) = 0.52
<br><br>As per my understanding, F1-score depends on both Precision and Recall. It is really important that both these are balanced values. Consider an example where Precision = 0.01 and Recall = 1.00
<br>F1-Score = (2 * 0.01 * 1) / (0.01 + 1) = 0.02
<br>This is a very low F1-Score.
<br>There might exists another good baseline classifier which has a Precision and Recall value of 1 and that gives us a F1-Score = (2 * 1 * 1) / (1 + 1) = 1 which is greater than 0.52 obtained previously.

#### Multinomial Naive Bayes Classifier with Unigrams and Bigrams.

#### Tasks
1) Creating a pipeline to train the Naive Bayes classifier model.
2) Fitting the training data using the classifier model.
3) Get the predictions and compute the Precision, Recall, and F1-Score for training and validation datasets.

In [9]:
#Creating the multinomial Naive Bayes classifier Pipeline:
#Including the unigrams and bigrams.
NB_pipeline = Pipeline([
    ('count_vectorizer', CountVectorizer(ngram_range=(1, 2),stop_words = ENGLISH_STOP_WORDS)),
    ('naive_bayes_classifier', MultinomialNB())
])

In [10]:
#Fitting the training data.
NB_pipeline.fit(train_text, train_label_code)

Pipeline(steps=[('count_vectorizer',
                 CountVectorizer(ngram_range=(1, 2),
                                 stop_words=frozenset({'a', 'about', 'above',
                                                       'across', 'after',
                                                       'afterwards', 'again',
                                                       'against', 'all',
                                                       'almost', 'alone',
                                                       'along', 'already',
                                                       'also', 'although',
                                                       'always', 'am', 'among',
                                                       'amongst', 'amoungst',
                                                       'amount', 'an', 'and',
                                                       'another', 'any',
                                                       'anyhow', 'anyone',

In [11]:
#Getting the predictions on training and validation datasets.
train_predictions = NB_pipeline.predict(train_text)
validation_predictions = NB_pipeline.predict(validation_text)

In [12]:
#Computing Precision, Recall, and F1-Score for Training data.
#Version 1.
#Since our clickbait data is labelled as '1' & we want to choose this class as target class. We use pos_label = 1.
train_precision = precision_score(train_label_code, train_predictions, pos_label=1)
train_recall = recall_score(train_label_code, train_predictions, pos_label=1)
train_f1_score = f1_score(train_label_code, train_predictions, pos_label=1)

In [13]:
#Printing the training data's Precision, Recall, and F1-Score.
print("Training Precision:", round(train_precision,2))
print("Training Recall:", round(train_recall,2))
print("Training F1-Score:", round(train_f1_score,2))

Training Precision: 0.99
Training Recall: 1.0
Training F1-Score: 0.99


In [14]:
#Computing Precision, Recall, and F1-Score for Validation data.
#Version 1.
#Since our clickbait data is labelled as '1' & we want to choose this class as target class. We use pos_label = 1.
validation_precision = precision_score(validation_label_code, validation_predictions, pos_label=1)
validation_recall = recall_score(validation_label_code, validation_predictions, pos_label=1)
validation_f1_score = f1_score(validation_label_code, validation_predictions, pos_label=1)

In [15]:
#Printing the validation data's Precision, Recall, and F1-Score.
print("Validation Precision:", round(validation_precision,2))
print("Validation Recall:", round(validation_recall,2))
print("Validation F1-Score:", round(validation_f1_score,2))

Validation Precision: 0.9
Validation Recall: 0.69
Validation F1-Score: 0.78


In [16]:
#Computing Precision, Recall, and F1-Score for Training data.
#Version 2.
print(classification_report(train_label_code, train_predictions, target_names=['Not Clickbait', 'Clickbait']))

               precision    recall  f1-score   support

Not Clickbait       1.00      1.00      1.00      1141
    Clickbait       0.99      1.00      0.99       578

     accuracy                           1.00      1719
    macro avg       1.00      1.00      1.00      1719
 weighted avg       1.00      1.00      1.00      1719



In [17]:
#Computing Precision, Recall, and F1-Score for Validation data.
#Version 2.
print(classification_report(validation_label_code, validation_predictions, target_names=['Not Clickbait', 'Clickbait']))

               precision    recall  f1-score   support

Not Clickbait       0.85      0.96      0.90       123
    Clickbait       0.90      0.69      0.78        68

     accuracy                           0.86       191
    macro avg       0.88      0.83      0.84       191
 weighted avg       0.87      0.86      0.86       191



<b>COMMENTS: From the above classification reports, we can see that the classifier model performs very well on training data with the metrics as Precision = 0.99, Recall = 1.00, F1-Score = 0.99, Accuracy = 1.00 [target class: Clickbait]. This is almost the ideal performance.
<br><br><b>Now, when we see the classifier's performance on validation data, it is also not bad. This can be verified from the metrics values as Precision = 0.90, Recall = 0.69, F1-Score = 0.78, Accuracy = 0.86 [target class: Clickbait]. Even though accuracy is high, we cannot just depend on it blindly, we need to consider the average recall and F1-Score. The imbalance in the dataset might be a reason for it because the metrics for Not Clickbait are better than Clickbait and we have more samples for Not Clickbait.

#### Hyperparameter Tuning

#### Tasks
1) Tune at least 3 hyperparameters to try to improve the model's overall performance.
2) Using the ParameterGrid class, performing a little grid search.
3) Getting the best metrics.

In [18]:
#Tuning the hyperparameters to improve the model's performance.
parameters_grid = {
    'count_vectorizer__max_df': [0.25, 0.5, 0.75, 1],
    'naive_bayes_classifier__alpha': [0.01, 0.1, 1.0, 10.0],
    'count_vectorizer__ngram_range': [(1, 1), (1, 2)]
}

In [19]:
#Using the ParameterGrid class,
#We perform a grid search to check if the model performs better at any other values of the hyperparameters.

#Initializing the best metrics as 0 or None.
best_f1_score = 0
best_parameters = None
best_report = None

#We repeat the fit and predict processes on different values of inputs as a grid search to get the best results.
for parameters in ParameterGrid(parameters_grid):
    NB_pipeline.set_params(**parameters)

    NB_pipeline.fit(train_text, train_label_code)

    validation_predictions = NB_pipeline.predict(validation_text)

    overall_report = classification_report(validation_label_code, validation_predictions, target_names=["Non-Clickbait", "Clickbait"], zero_division = 0)
    validation_f1_score = f1_score(validation_label_code, validation_predictions, pos_label=1)

    print("Parameters:", parameters)
    print("Validation F1-Score:", round(validation_f1_score,2))
    print(overall_report)
    print()

    if validation_f1_score > best_f1_score:
        best_f1_score = validation_f1_score
        best_parameters = parameters
        best_report = overall_report

Parameters: {'count_vectorizer__max_df': 0.25, 'count_vectorizer__ngram_range': (1, 1), 'naive_bayes_classifier__alpha': 0.01}
Validation F1-Score: 0.77
               precision    recall  f1-score   support

Non-Clickbait       0.85      0.93      0.89       123
    Clickbait       0.84      0.71      0.77        68

     accuracy                           0.85       191
    macro avg       0.85      0.82      0.83       191
 weighted avg       0.85      0.85      0.84       191


Parameters: {'count_vectorizer__max_df': 0.25, 'count_vectorizer__ngram_range': (1, 1), 'naive_bayes_classifier__alpha': 0.1}
Validation F1-Score: 0.79
               precision    recall  f1-score   support

Non-Clickbait       0.88      0.91      0.89       123
    Clickbait       0.83      0.76      0.79        68

     accuracy                           0.86       191
    macro avg       0.85      0.84      0.84       191
 weighted avg       0.86      0.86      0.86       191


Parameters: {'count_vectori

Parameters: {'count_vectorizer__max_df': 0.75, 'count_vectorizer__ngram_range': (1, 1), 'naive_bayes_classifier__alpha': 1.0}
Validation F1-Score: 0.79
               precision    recall  f1-score   support

Non-Clickbait       0.88      0.91      0.89       123
    Clickbait       0.83      0.76      0.79        68

     accuracy                           0.86       191
    macro avg       0.85      0.84      0.84       191
 weighted avg       0.86      0.86      0.86       191


Parameters: {'count_vectorizer__max_df': 0.75, 'count_vectorizer__ngram_range': (1, 1), 'naive_bayes_classifier__alpha': 10.0}
Validation F1-Score: 0.58
               precision    recall  f1-score   support

Non-Clickbait       0.75      0.99      0.86       123
    Clickbait       0.97      0.41      0.58        68

     accuracy                           0.79       191
    macro avg       0.86      0.70      0.72       191
 weighted avg       0.83      0.79      0.76       191


Parameters: {'count_vectori

In [20]:
#Getting the best metrics.
print("Best Parameters:", best_parameters)
print("\nBest Validation F1-Score:", round(best_f1_score,2))
print("\nBest Validation Metrics:")
print(best_report)

Best Parameters: {'count_vectorizer__max_df': 0.25, 'count_vectorizer__ngram_range': (1, 1), 'naive_bayes_classifier__alpha': 0.1}

Best Validation F1-Score: 0.79

Best Validation Metrics:
               precision    recall  f1-score   support

Non-Clickbait       0.88      0.91      0.89       123
    Clickbait       0.83      0.76      0.79        68

     accuracy                           0.86       191
    macro avg       0.85      0.84      0.84       191
 weighted avg       0.86      0.86      0.86       191



<b>COMMENTS: Before Hyperparameter tuning, the validation dataset metrics were as follows:
<br><b>Precision = 0.90, Recall = 0.69, F1-Score = 0.78, Accuracy = 0.86 [target class: Clickbait]
<br><br><b>After Hyperparameter tuning, the validation dataset metrics that we found for the best model using parameter grid are as follows:
<br><b>Precision = 0.83, Recall = 0.76, F1-Score = 0.79, Accuracy = 0.86 [target class: Clickbait]
<br><br><b>This clearly shows that the hyperparameter tuning did an improvement in Recall and F1-score. A more improvement was expected. However, any improvement is an improvement.

#### Model Selection

#### Tasks
1) Using the best model parameters, we will select this as our model and fit our training data.
2) Make predictions on test data.

In [21]:
#Use the best parameters as our parameters in model pipeline.
NB_pipeline.set_params(**best_parameters)

Pipeline(steps=[('count_vectorizer',
                 CountVectorizer(max_df=0.25,
                                 stop_words=frozenset({'a', 'about', 'above',
                                                       'across', 'after',
                                                       'afterwards', 'again',
                                                       'against', 'all',
                                                       'almost', 'alone',
                                                       'along', 'already',
                                                       'also', 'although',
                                                       'always', 'am', 'among',
                                                       'amongst', 'amoungst',
                                                       'amount', 'an', 'and',
                                                       'another', 'any',
                                                       'anyhow', 'anyone',
      

In [22]:
#Fitting the classifier model on the training dataset with the best parameters.
NB_pipeline.fit(train_text, train_label_code)

Pipeline(steps=[('count_vectorizer',
                 CountVectorizer(max_df=0.25,
                                 stop_words=frozenset({'a', 'about', 'above',
                                                       'across', 'after',
                                                       'afterwards', 'again',
                                                       'against', 'all',
                                                       'almost', 'alone',
                                                       'along', 'already',
                                                       'also', 'although',
                                                       'always', 'am', 'among',
                                                       'amongst', 'amoungst',
                                                       'amount', 'an', 'and',
                                                       'another', 'any',
                                                       'anyhow', 'anyone',
      

In [23]:
#Making Predictions on the test dataset.
test_predictions = NB_pipeline.predict(test_text)

In [24]:
#Computing Precision, Recall, and F1-Score for Testing data.
#Version 1.
#Since our clickbait data is labelled as '1' & we want to choose this class as target class. We use pos_label = 1.
test_precision = precision_score(test_label_code, test_predictions, pos_label=1)
test_recall = recall_score(test_label_code, test_predictions, pos_label=1)
test_f1_score = f1_score(test_label_code, test_predictions, pos_label=1)

In [25]:
#Printing the test data's Precision, Recall, and F1-Score.
print("Testing Precision:", round(test_precision,2))
print("Testing Recall:", round(test_recall,2))
print("Testing F1-Score:", round(test_f1_score,2))

Testing Precision: 0.84
Testing Recall: 0.78
Testing F1-Score: 0.81


In [26]:
#Computing Precision, Recall, and F1-Score for Testing data.
#Version 2.
print(classification_report(test_label_code, test_predictions, target_names=['Not Clickbait', 'Clickbait']))

               precision    recall  f1-score   support

Not Clickbait       0.89      0.92      0.90       310
    Clickbait       0.84      0.78      0.81       168

     accuracy                           0.87       478
    macro avg       0.86      0.85      0.86       478
 weighted avg       0.87      0.87      0.87       478



<b>COMMENTS: On evaluating this model's performance on Test data, we can see that the model performs better on the test data as compared to the validation data. As long as the metrics of precision and recall are high and almost equal, they indicate a good model.
<br><br><b>The model's metrics on test data are as follows:
<br><b>Precision = 0.84, Recall = 0.78, F1-Score = 0.81, Accuracy = 0.87 [target class: Clickbait]

#### Key Indicators

#### Tasks
1) Now, we want to find the words which our machine identified as the key indicators of our target class "clickbait".
2) Using the log probabilities, find the top 5 key indicators of clickbait.

In [27]:
#Getting the list of words/features from our vectorizer in the pipeline.
feature_names = NB_pipeline.named_steps['count_vectorizer'].get_feature_names_out()

In [28]:
#Getting the log probabilities for the indicators of "clickbait" (target class)
#from the naive bayes classifier in the pipeline.
log_probabilities = NB_pipeline.named_steps['naive_bayes_classifier'].feature_log_prob_[1]

In [29]:
#Forming a key-value pair of words/features and their log probabilities to get the top 5.
features_log_probabilities = dict(zip(feature_names, log_probabilities))

In [30]:
#Sorting the dictionary in descending order based on the log probabilities of indicators.
sorted_features = sorted(features_log_probabilities.items(), key=lambda x: x[1], reverse=True)

#Selecting the top 5 words as strong clickbait indicators.
top_clickbait_indicators = [indicator for indicator, log_probability in sorted_features[:5]]

In [31]:
#Printing the top 5 clickbait indicators we identified from the dictionary.
print("The Top 5 Key Indicators Of Clickbait Are:")
for s_no, indicator in enumerate(top_clickbait_indicators, 1):
    print(f"{s_no}. {indicator}")

The Top 5 Key Indicators Of Clickbait Are:
1. believe
2. won
3. ll
4. new
5. guess


<b>COMMENTS: A general speculation of the data gave a little bit idea about why these words might have been identified by the model as the key indicators.
<br><br><b>believe: This word was used as a clickbait to trap a lot of people by providing them impossible/unBELIVable deals, hence the use of word 'believe' was used a lot as "won't believe" which means unbelivable.
<br><br><b>won: This word was used as a clickbait to trap a lot of people by either telling them that they have WON something or providing them impossible/unbelivable deals, hence the use of word "won't" was used a lot as "won't believe" which means unbelivable.
<br><br><b>ll: This word was not used directly as a clickbait but it was a part of a contaction which we often use in English like We'll, you'll. Since this kind of text was present a lot of times in the data where the scammer addressed the victim say as "You'll never believe" which caused this word/token to show up with top 5 log probabilities.
<br><br><b>new: This word was used as a clickbait to trap a lot of people by either telling them that they have won something new, hence the use of word "new" was used a lot as "won new PS5", etc.
<br><br><b>guess: This word was used as a clickbait to trap a lot of people by providing them impossible/unbelivable deals, hence the use of word "guess" was used a lot as "never guess" which means something shocking which tempts the victim to fall in the trap of clickbait.

#### Regular Expressions

#### Tasks
1) This is a fun and interesting part. Rather than using the classifier, we will use a regex in this problem and in that regex we will use the top 5 indicators of the target class as our hits for clickbait.
2) If anyone of the key indicator is present in our text sentence, we mark it as our target class else not the target class.
3) Compute the metrics for this.

In [32]:
#Creating a regular expression pattern that allows to search for a regular expression match object,
#with any of the top 5 indicators we found in last step.
pattern = r'\b(?:' + '|'.join(re.escape(indicator) for indicator in top_clickbait_indicators) + r')\b'

#Function to search if top keyword exists in the text using the regex pattern (IGNORING THE CASE).
def top_indicator_detector(text):
    return re.search(pattern, text, flags=re.IGNORECASE) is not None

#Applying the function on the test set.
test_predictions = (test_text.apply(top_indicator_detector)).astype(int)

In [33]:
#Computing Precision, Recall, and F1-Score for Testing data.
#Version 1.
#Since our clickbait data is labelled as '1' & we want to choose this class as target class. We use pos_label = 1.
test_precision = precision_score(test_label_code, test_predictions, pos_label=1)
test_recall = recall_score(test_label_code, test_predictions, pos_label=1)
test_f1_score = f1_score(test_label_code, test_predictions, pos_label=1)

In [34]:
#Printing the test data's Precision, Recall, and F1-Score.
print("Testing Precision:", round(test_precision,2))
print("Testing Recall:", round(test_recall,2))
print("Testing F1-Score:", round(test_f1_score,2))

Testing Precision: 0.63
Testing Recall: 0.16
Testing F1-Score: 0.26


In [35]:
#Computing Precision, Recall, and F1-Score for Testing data.
#Version 2.
print(classification_report(test_label_code, test_predictions, target_names=['Not Clickbait', 'Clickbait']))

               precision    recall  f1-score   support

Not Clickbait       0.68      0.95      0.79       310
    Clickbait       0.63      0.16      0.26       168

     accuracy                           0.67       478
    macro avg       0.65      0.55      0.52       478
 weighted avg       0.66      0.67      0.60       478



<b>COMMENTS: When we tried this fun experiment which uses regex to identify the text as clickbait or not, we can see that it performs as a low to average classifier. It is better than a random classifier which might have an accuracy of 50%. However, the recall rate and F1-Score is very low. It is expected as we are just using top 5 predictors. Also, the cases with contractions created problem, the cases with New Jersey, New York created problem due the indicator word "new". There might be other problems as well which might not be visible directly but could be found when delved deeper into this.
<br><br><b>The metrics we get for this are as follows:
<br><b>Precision = 0.63, Recall = 0.16, F1-Score = 0.26, Accuracy = 0.67 [target class Clickbait]

#### Compairing Results obtained so far

When we compare the rules based classifier with our machine learning solution, we can clearly see from the metrics that our machine-learning solution perform the best.
<br>Rules Based: Precision = 0.63, Recall = 0.16, F1-Score = 0.26, Accuracy = 0.67
<br>Machine-learning Based: Precision = 0.84, Recall = 0.78, F1-Score = 0.81, Accuracy = 0.87
<br><br>The reason it perform better is that machine learning solution uses the count vectorizer and considers each word into its training and improving. The rules based model just uses top 5 words to identify the target class. The performance is not that bad when we think that with just 5 words, we got a precision of 0.63 and accuracy of 0.67. But the machine learning classifier performs way better than the rules based solution.
<br>When we compare the baseline classifier with the other two models, the baseline classifier and the rules based classifier are both low grade classifiers as one gives high precision and the other gives high recall, whereas we are finding the sweetspot where the precision and recall are high and almost equal.
<br><br>Rules Based: Precision = 0.63, Recall = 0.16, F1-Score = 0.26, Accuracy = 0.67
<br>Machine-learning Based: Precision = 0.84, Recall = 0.78, F1-Score = 0.81, Accuracy = 0.87
<br>Baseline classifier: Precision = 0.35, Recall = 1, F1-Score = 0.51, Accuracy = 0.35